In [1]:
!pip install openai -q

In [2]:
import sys, os, warnings, re, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split

SEED = 42

INPUT_PATH = '/kaggle/input/translit-datasets/'

In [3]:
from openai import OpenAI
from kaggle_secrets import UserSecretsClient

secret = UserSecretsClient().get_secret("openai")

client=OpenAI(
    api_key = secret
)

# Dataset

In [8]:
# Insert appropriate dataset reading code from dataset_readers.py

def read_dataset():
    pass

prompt = '''INSERT PROMPT HERE'''

In [ ]:
train_df, test_df, label_names, dataset_name = read_dataset()

print(prompt)

pd.set_option('max_colwidth', 200)
display(train_df.head())
display(test_df.head())

print(f'{len(train_df)=}, {len(test_df)=}')
print(label_names)

# Prediction code

In [10]:
def predict_with_chatgpt(df, prompt, text_col = 'sentence'):
    
    def chat_with_chatgpt(prompt):
        chat_completion=client.chat.completions.create(
            messages=[
                {"role":"user",
                 "content":prompt
                }
            ],
            model="gpt-3.5-turbo"
        )
        message = chat_completion.choices[0].message.content
        return message
    
    # Get chatbot response via API
    text = []
    for index, row in enumerate(df['sentence']):
        chatbot_response = chat_with_chatgpt(prompt.format(row.lower()))
        text.append(chatbot_response)
        
        if index % 100 == 0:
            print(index)
            
    return text
   
    
def parse_output(df, text):
    # Parse output
    dfs = []
    for item in text:
        if '<pred>' not in item:
            pred = ''
        else:
            parts = item.split('<pred>')
            pred  = parts[1].split('</pred>')[0].strip()
        
        pred_tmp_df = pd.DataFrame({'pred': [pred.lower()]})
        dfs.append( pred_tmp_df )
    
    pred_df = pd.concat(dfs, ignore_index=True)
    
    df['pred'] = pd.Categorical(pred_df.pred)
    df.to_csv('/kaggle/working/pred_df.csv', index=False)

    return df

# Predict

In [ ]:
df = test_df

txt = predict_with_chatgpt(df, prompt, 'translated')

0


In [ ]:
df_out = parse_output(df, txt)

display(df_out.label.value_counts())
display(df_out.pred.value_counts())

In [ ]:
## Do manual cleanup as required

# df_out.pred = df.pred.cat.rename_categories({
#     'neutral': 'positive',
#     '': 'positive'
# })
# df_out.pred = df_out.pred.cat.remove_unused_categories()

# df = pd.read_csv(
#     '/kaggle/working/pred_df.csv', 
#     dtype={'sentence': 'string', 'label': 'category', 'pred': 'category'}
# )


# df.pred = df.pred.replace({
#    'neutral': 'positive',
# })
# df.pred = df.pred.cat.remove_unused_categories()

# df.to_csv('/kaggle/working/pred_df.csv', index=False)


# display(df.label.value_counts())
# display(df.pred.value_counts())

# display(df_out.pred.value_counts())

In [ ]:
y_test, y_pred = df_out.label.cat.codes, df_out.pred.cat.codes

print(classification_report(
    y_test, y_pred, digits=5, target_names=label_names, 
))
ConfusionMatrixDisplay.from_predictions(
    y_test, y_pred, display_labels=label_names
)